In [19]:
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [23]:
df = pd.read_csv("/content/drive/MyDrive/data/smart_watch/smartwatch_gestures.csv")

In [24]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [25]:
df.head()

,user,gesture,trial,step,sys_current_time,sys_nanotime,event_timestamp,X,Y,Z
0,U08,11,20,0,1.387456e+12,6.586995e+14,1.872000e+11,-1.379060,0.919373,9.500193
1,U08,11,20,1,1.387456e+12,6.586996e+14,1.873100e+11,-0.306458,2.758120,10.266337
2,U08,11,20,2,1.387456e+12,6.586997e+14,1.874200e+11,1.685518,6.435614,10.113108
3,U08,11,20,3,1.387456e+12,6.586998e+14,1.875300e+11,0.919373,1.838747,9.500193
4,U08,11,20,4,1.387456e+12,6.586999e+14,1.876400e+11,-0.919373,-5.209783,10.266337


In [26]:
df['user'].value_counts()['U01']

5193

In [50]:
# find minimum data series size for earch gesture trial
min_trial_len = []

for i in range(1, 21):
  x = df.loc[df["gesture"] == i].groupby(["user", "gesture", "trial"]).size().min()
  min_trial_len.append(x)

min_trial_len

[11,
 11,
 11,
 10,
 16,
 16,
 18,
 19,
 14,
 13,
 14,
 13,
 14,
 13,
 14,
 14,
 16,
 14,
 16,
 15]

In [71]:
# Minimum trial size is 10.  Standardize on first 10 time slices

temp_df = df.copy()

for i in range(0, len(min_trial_len)):
  # indexes = temp_df[(temp_df.gesture == i+1) & (temp_df.step > min_trial_len[i]-1)].index
  indexes = temp_df[(temp_df.gesture == i+1) & (temp_df.step > 9)].index
  temp_df.drop(indexes, inplace=True)


In [72]:
max_trial_len = []

for i in range(1, 21):
  x = temp_df.loc[df["gesture"] == i].groupby(["user", "gesture", "trial"]).size().max()
  max_trial_len.append(x)

max_trial_len

[10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10]

In [73]:
df = temp_df.copy()

In [74]:
df.head(20)

,user,gesture,trial,step,sys_current_time,sys_nanotime,event_timestamp,X,Y,Z
0,U08,11,20,0,1.387456e+12,6.586995e+14,1.872000e+11,-1.379060,0.919373,9.500193
1,U08,11,20,1,1.387456e+12,6.586996e+14,1.873100e+11,-0.306458,2.758120,10.266337
2,U08,11,20,2,1.387456e+12,6.586997e+14,1.874200e+11,1.685518,6.435614,10.113108
3,U08,11,20,3,1.387456e+12,6.586998e+14,1.875300e+11,0.919373,1.838747,9.500193
4,U08,11,20,4,1.387456e+12,6.586999e+14,1.876400e+11,-0.919373,-5.209783,10.266337
5,U08,11,20,5,1.387456e+12,6.587001e+14,1.877500e+11,-0.459687,-4.903325,6.588843
6,U08,11,20,6,1.387456e+12,6.587002e+14,1.878600e+11,1.379060,-2.145205,1.072602
7,U08,11,20,7,1.387456e+12,6.587003e+14,1.879700e+11,4.290410,-3.217807,5.975927
8,U08,11,20,8,1.387456e+12,6.587004e+14,1.880800e+11,10.879252,-1.072602,16.701952
9,U08,11,20,9,1.387456e+12,6.587005e+14,1.881900e+11,7.967903,-3.830723,13.790602
